In [3]:
import optuna
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [4]:
data = pd.read_csv("dataset\Admission_Predict.csv")

In [5]:
data.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         400 non-null    int64  
 1   GRE Score          400 non-null    int64  
 2   TOEFL Score        400 non-null    int64  
 3   University Rating  400 non-null    int64  
 4   SOP                400 non-null    float64
 5   LOR                400 non-null    float64
 6   CGPA               400 non-null    float64
 7   Research           400 non-null    int64  
 8   Chance of Admit    400 non-null    float64
dtypes: float64(4), int64(5)
memory usage: 28.2 KB


In [7]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Serial No.,400.0,200.500000,115.614301,1.00,100.75,200.50,300.2500,400.00
GRE Score,400.0,316.807500,11.473646,290.00,308.00,317.00,325.0000,340.00
TOEFL Score,400.0,107.410000,6.069514,92.00,103.00,107.00,112.0000,120.00
University Rating,400.0,3.087500,1.143728,1.00,2.00,3.00,4.0000,5.00
SOP,400.0,3.400000,1.006869,1.00,2.50,3.50,4.0000,5.00
LOR,400.0,3.452500,0.898478,1.00,3.00,3.50,4.0000,5.00
CGPA,400.0,8.598925,0.596317,6.80,8.17,8.61,9.0625,9.92
Research,400.0,0.547500,0.498362,0.00,0.00,1.00,1.0000,1.00
Chance of Admit,400.0,0.724350,0.142609,0.34,0.64,0.73,0.8300,0.97


In [8]:
data.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [10]:
X = data.drop(["Serial No.", "Chance of Admit "], axis=1)

In [12]:
y = data["Chance of Admit "]

In [13]:
y.head()

0    0.92
1    0.76
2    0.72
3    0.80
4    0.65
Name: Chance of Admit , dtype: float64

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=25)

In [15]:
sc = StandardScaler()

In [18]:
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

In [30]:
def objective(trail, data=X, target=y):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.25, random_state=25)
    param = {
        "tree_method": "auto", # {'approx', 'auto', 'exact', 'gpu_hist', 'hist'}
        "lambda": trail.suggest_loguniform('lambda', 1e-4, 10.0),
        "alpha": trail.suggest_loguniform('alpha', 1e-4, 10.0),
        "colsample_bytree": trail.suggest_categorical('colsample_bytree', [.1, .2, .3, .4, .5, .6, .7, .8, .9, 1]),
        "subsample": trail.suggest_categorical('subsample', [.1, .2, .3, .4, .5, .6, .7, .8, .9, 1]),
        "learning_rate": trail.suggest_categorical('learning_rate', [.00001, .0003, .008, .02, .01, 1, 8]),
        "n_estimators": 3000,
        "max_depth": trail.suggest_categorical('max_depth', [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),
        "random_state": trail.suggest_categorical('random_state', [10, 20, 30, 2000, 3454, 243123]),
        "min_child_weight": trail.suggest_int("min_child_weight", 1, 200)
    }

    model = xgb.XGBRegressor(**param)
    model.fit(train_x, train_y, eval_set=[(test_x, test_y)], verbose=True)
    pred = model.predict(test_x)
    mse = mean_squared_error(test_y, pred)
    return mse

In [31]:
find_params=optuna.create_study()
find_params.optimize(objective, n_trials=10)
find_params.best_trial.params

[I 2024-08-03 12:42:16,888] A new study created in memory with name: no-name-ab4e3d1d-cc51-4b15-8cf5-87d40e24e128


[0]	validation_0-rmse:0.15112
[1]	validation_0-rmse:0.15028
[2]	validation_0-rmse:0.14948
[3]	validation_0-rmse:0.14898
[4]	validation_0-rmse:0.14830
[5]	validation_0-rmse:0.14772


C:\Users\BoSS\AppData\Local\Temp\ipykernel_3764\3707086658.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "lambda": trail.suggest_loguniform('lambda', 1e-4, 10.0),
C:\Users\BoSS\AppData\Local\Temp\ipykernel_3764\3707086658.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "alpha": trail.suggest_loguniform('alpha', 1e-4, 10.0),


[6]	validation_0-rmse:0.14716
[7]	validation_0-rmse:0.14655
[8]	validation_0-rmse:0.14600
[9]	validation_0-rmse:0.14534
[10]	validation_0-rmse:0.14473
[11]	validation_0-rmse:0.14395
[12]	validation_0-rmse:0.14329
[13]	validation_0-rmse:0.14260
[14]	validation_0-rmse:0.14205
[15]	validation_0-rmse:0.14136
[16]	validation_0-rmse:0.14082
[17]	validation_0-rmse:0.14019
[18]	validation_0-rmse:0.13949
[19]	validation_0-rmse:0.13899
[20]	validation_0-rmse:0.13836
[21]	validation_0-rmse:0.13771
[22]	validation_0-rmse:0.13707
[23]	validation_0-rmse:0.13659
[24]	validation_0-rmse:0.13600
[25]	validation_0-rmse:0.13543
[26]	validation_0-rmse:0.13484
[27]	validation_0-rmse:0.13420
[28]	validation_0-rmse:0.13366
[29]	validation_0-rmse:0.13307
[30]	validation_0-rmse:0.13258
[31]	validation_0-rmse:0.13206
[32]	validation_0-rmse:0.13162
[33]	validation_0-rmse:0.13123
[34]	validation_0-rmse:0.13064
[35]	validation_0-rmse:0.13004
[36]	validation_0-rmse:0.12955
[37]	validation_0-rmse:0.12905
[38]	validat

[I 2024-08-03 12:43:10,323] Trial 0 finished with value: 0.0049207508572086025 and parameters: {'lambda': 0.04484249268881329, 'alpha': 0.004967066999970753, 'colsample_bytree': 0.7, 'subsample': 0.5, 'learning_rate': 0.008, 'max_depth': 4, 'random_state': 10, 'min_child_weight': 35}. Best is trial 0 with value: 0.0049207508572086025.


...

[0]	validation_0-rmse:0.15173
[1]	validation_0-rmse:0.15173
[2]	validation_0-rmse:0.15173
[3]	validation_0-rmse:0.15173
[4]	validation_0-rmse:0.15173
[5]	validation_0-rmse:0.15173


C:\Users\BoSS\AppData\Local\Temp\ipykernel_3764\3707086658.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "lambda": trail.suggest_loguniform('lambda', 1e-4, 10.0),
C:\Users\BoSS\AppData\Local\Temp\ipykernel_3764\3707086658.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "alpha": trail.suggest_loguniform('alpha', 1e-4, 10.0),


[6]	validation_0-rmse:0.15173
[7]	validation_0-rmse:0.15173
[8]	validation_0-rmse:0.15173
[9]	validation_0-rmse:0.15173
[10]	validation_0-rmse:0.15173
[11]	validation_0-rmse:0.15173
[12]	validation_0-rmse:0.15173
[13]	validation_0-rmse:0.15173
[14]	validation_0-rmse:0.15173
[15]	validation_0-rmse:0.15173
[16]	validation_0-rmse:0.15173
[17]	validation_0-rmse:0.15173
[18]	validation_0-rmse:0.15173
[19]	validation_0-rmse:0.15173
[20]	validation_0-rmse:0.15173
[21]	validation_0-rmse:0.15173
[22]	validation_0-rmse:0.15173
[23]	validation_0-rmse:0.15173
[24]	validation_0-rmse:0.15173
[25]	validation_0-rmse:0.15173
[26]	validation_0-rmse:0.15173
[27]	validation_0-rmse:0.15173
[28]	validation_0-rmse:0.15173
[29]	validation_0-rmse:0.15173
[30]	validation_0-rmse:0.15173
[31]	validation_0-rmse:0.15173
[32]	validation_0-rmse:0.15173
[33]	validation_0-rmse:0.15173
[34]	validation_0-rmse:0.15173
[35]	validation_0-rmse:0.15173
[36]	validation_0-rmse:0.15173
[37]	validation_0-rmse:0.15173
[38]	validat

[I 2024-08-03 12:52:40,184] Trial 9 finished with value: 0.023024072334074976 and parameters: {'lambda': 0.004662083450761026, 'alpha': 0.05949985666780798, 'colsample_bytree': 0.5, 'subsample': 0.5, 'learning_rate': 0.01, 'max_depth': 8, 'random_state': 30, 'min_child_weight': 143}. Best is trial 7 with value: 0.004683172723299316.


{'lambda': 0.7348079202263718,
 'alpha': 0.025518117492534194,
 'colsample_bytree': 0.4,
 'subsample': 1,
 'learning_rate': 0.008,
 'max_depth': 12,
 'random_state': 243123,
 'min_child_weight': 51}

In [45]:
best_params1 = {
    'lambda': 1.10382994152563448,
    'alpha' : 0.2002246174362676,
    'colsample_bytree': 0.3,
    'subsample': 0.9,
    'learning_rate': 0.008,
    'max_depth': 4,
    'random_state':3454,
    'min_child_weight': 30
}

In [46]:
best_params = {'lambda': 0.7348079202263718,
 'alpha': 0.025518117492534194,
 'colsample_bytree': 0.4,
 'subsample': 1,
 'learning_rate': 0.008,
 'max_depth': 12,
 'random_state': 243123,
 'min_child_weight': 51}

In [42]:
from sklearn.metrics import r2_score

In [49]:
model = xgb.XGBRegressor(**best_params)
model.fit(X_train_sc, y_train)
pred_train = model.predict(X_train_sc)
pred_test = model.predict(X_test_sc)
mse_train = mean_squared_error(y_train, pred_train)
mse_test = mean_squared_error(y_test, pred_test)
print("MSE train:", mse_train)
print("MSE test:", mse_test)
print('-'*25)
print("RMSE train:", np.sqrt(mse_train))
print("RMSE test:", np.sqrt(mse_test))
print('-'*25)
print("R2 train:", r2_score(y_train, pred_train))
print("R2 test:", r2_score(y_test, pred_test))

MSE train: 0.008482528591370062
MSE test: 0.010799997029211543
-------------------------
RMSE train: 0.09210064381626255
RMSE test: 0.10392303416091903
-------------------------
R2 train: 0.5621874529180377
R2 test: 0.5308624388725951


In [50]:
from sklearn.ensemble import RandomForestRegressor

In [51]:
model = RandomForestRegressor()
model.fit(X_train_sc, y_train)
pred_train = model.predict(X_train_sc)
pred_test = model.predict(X_test_sc)
mse_train = mean_squared_error(y_train, pred_train)
mse_test = mean_squared_error(y_test, pred_test)
print("MSE train:", mse_train)
print("MSE test:", mse_test)
print('-'*25)
print("RMSE train:", np.sqrt(mse_train))
print("RMSE test:", np.sqrt(mse_test))
print('-'*25)
print("R2 train:", r2_score(y_train, pred_train))
print("R2 test:", r2_score(y_test, pred_test))

MSE train: 0.0006682105666666666
MSE test: 0.004915453900000006
-------------------------
RMSE train: 0.025849769180142916
RMSE test: 0.07011029810234731
-------------------------
R2 train: 0.9655113487649132
R2 test: 0.7864791954809875
